In [1]:
#!/home/xyang18/miniconda3/envs/pytorch/bin/ python
# -*- coding: utf-8 -*-
# Python version: 3.6

import os
import sys
import copy
import time
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import WeightedRandomSampler, TensorDataset
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, BatchNorm1d, Dropout, Flatten, BCELoss
from torch.optim import Adam, SGD
from torch import nn
# from torchsummary import summary
# from torch.utils.tensorboard import SummaryWriter

from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import sklearn.metrics as metrics


In [2]:
from matplotlib import pyplot as plt
%matplotlib inline

In [3]:
gpu_id=0
seed=5

In [4]:
import math

In [5]:
if gpu_id>=0:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    cuda_id = "cuda:" + str(0)  # cuda:2

device = torch.device(cuda_id if torch.cuda.is_available() else "cpu")
print("Device:", device)
if (torch.cuda.is_available()):
    torch.cuda.set_device(cuda_id)
    print("Current GPU ID:", torch.cuda.current_device())

Device: cuda:0
Current GPU ID: 0


In [6]:

X_total = np.load('hhar_time_X.npy').astype('float32')
y_total = np.load('hhar_time_y.npy').astype(int)

X_total=np.nan_to_num(X_total)
for i in range(X_total.shape[1]):
    # print(type(i))
    
    ch_data = X_total[:,i,:] # the data of channel id
    scaler = MinMaxScaler() # maybe different scalers?
    ch_data = scaler.fit_transform(ch_data) # scale the data in this channel to [0,1]
    X_total[:,i,:] = ch_data # assign normalized data to normalized_X
    
    # ch_max = np.max(X_total[:,i,:])
    # ch_min = np.min(X_total[:,i,:])
    # # print(ch_max, ch_min)
    # X_total[:,i,:] = (X_total[:,i,:] - ch_min) / (ch_max - ch_min)
    
X_total = np.transpose(X_total, (0,2,1))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=seed)

In [7]:
class HARModel(nn.Module):
    
    def __init__(self, n_sensor_channels=6, len_seq=100, n_hidden=128, n_layers=1, n_filters=64, 
                 n_classes=6, filter_size=(1,5), drop_prob=0.5):
        super(HARModel, self).__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_filters = n_filters
        self.n_classes = n_classes
        self.filter_size = filter_size
        self.n_sensor_channels = n_sensor_channels
        self.len_seq = len_seq

             
        self.conv1 = nn.Conv2d(1, n_filters, filter_size)
        self.conv2 = nn.Conv2d(n_filters, n_filters, filter_size)
        self.conv3 = nn.Conv2d(n_filters, n_filters, filter_size)
        self.conv4 = nn.Conv2d(n_filters, n_filters, filter_size)
        
        # self.fc0 = nn.Linear(n_sensor_channels*n_filters,n_sensor_channels*4)
        
        # self.lstm1  = nn.LSTM(64, n_hidden, n_layers)
        # self.lstm2  = nn.LSTM(n_hidden, n_hidden, n_layers)
        # self.multihead_attn = nn.MultiheadAttention(embed_dim=n_sensor_channels*4, num_heads=1) # 7232=113*64
        self.multihead_attn = nn.MultiheadAttention(embed_dim=n_sensor_channels*n_filters, num_heads=1) # 7232=113*64
        # self.multihead_attn = nn.MultiheadAttention(embed_dim=n_sensor_channels, num_heads=1) # 7232=113*64
        
        # self.fc = nn.Linear(2712, n_classes) #57856 = 8*113*64
        # self.fc = nn.Linear(n_sensor_channels*n_filters*(len_seq-0*(filter_size[1]-1)), n_classes) #57856 = 8*113*64
        self.fc = nn.Linear(n_sensor_channels*n_filters*(len_seq-4*(filter_size[1]-1)), n_classes) #57856 = 8*113*64

        self.dropout = nn.Dropout(drop_prob)
        # self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        # x = x.view(-1, NB_SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH,1) # for direct channel_gate
        # batch_size = x.shape[0]

        # x = x.view(-1, NB_SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH) # for deepconvlstm conv layers
        # x = torch.permute(x,(1,0,2))
        # print(x.shape)
        # x, attn_output_weights = self.multihead_attn0(x,x,x)

        # print(x.shape)
        # x = torch.permute(x,(2,1,0))
        # print(x.shape)
        # x = x.view(-1, 1, NB_SENSOR_CHANNELS, SLIDING_WINDOW_LENGTH) # draft
        x = torch.permute(x, (0,2,1))
        x = torch.unsqueeze(x, dim=1)
        # print(x.shape)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x)) # [64, 113, 8]
        
        # x = x.view(-1, NB_SENSOR_CHANNELS, 8, 1)
        # x = x.view(x.shape[0], x.shape[1], x.shape[2], 1)
        # x = x.view(x.shape[0], -1, 8)
        
        # print(x.shape)
        x = torch.permute(x, (3,0,1,2))
        x = x.view(x.shape[0], x.shape[1],-1)
        
        # print(x.shape)
        # x = x.view(8, x.shape[0], -1) # bak
        # x = F.relu(self.fc0(x))
        
    
        x, attn_output_weights = self.multihead_attn(x,x,x)
        x = self.dropout(x)
        x = F.relu(x)
#         x, attn_output_weights = self.multihead_attn1(x,x,x)
#         # x = self.dropout(x)
#         x = F.relu(x)    
        
        x = torch.permute(x, (1,0,2))
        
        # x, hidden = self.lstm1(x, hidden)
        # # x = self.dropout(x)
        # x, hidden = self.lstm2(x, hidden)
        # x = self.dropout(x)
        
        # x = x.contiguous().view(-1, self.n_hidden)

        x = torch.reshape(x, (x.shape[0],-1))
        # x = F.relu(self.fc0(x))
        # x = self.dropout(x)
        x = self.fc(x)
        
        return x
    
net = HARModel()

In [8]:
def train(net, epochs=600, batch_size=64, lr=0.001):
    # opt = torch.optim.Adam(net.parameters(), lr=lr)
    opt = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
    # opt = torch.optim.RMSprop(net.parameters(), lr=lr, momentum=0.1)
    # opt = torch.optim.SGD(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    # criterion = nn.BCEWithLogitsLoss()
    
    train_dataset = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
    train_loader = torch.utils.data.DataLoader(train_dataset,
        batch_size=batch_size, shuffle=True, drop_last = True)  

    test_dataset = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))
    test_loader = torch.utils.data.DataLoader(test_dataset,
        batch_size=batch_size, shuffle=False, drop_last = True) 
    
    if(train_on_gpu):
        net.cuda()
     
    for e in range(epochs):
        
        # initialize hidden state
        # h = net.init_hidden(batch_size)         
        train_losses = []    
        net.train()
        # for batch in iterate_minibatches(X_train, y_train, batch_size):
        for batch in train_loader:
            x, y = batch

            # inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            inputs, targets = x.to(device), y.to(device)  

            # if(train_on_gpu):
            #         inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            # h = tuple([each.data for each in h])
            
            # h = h[0].reshape((batch_size, -1)) # for GRU
            
            # zero accumulated gradients
            opt.zero_grad()   
            
            # get the output from the model
            output = net(inputs)
            # loss = criterion(output, torch.from_numpy(to_categorical(y, num_classes=NUM_CLASSES)).to(device))
            loss = criterion(output, torch.argmax(targets,dim=1))
            # print(output.shape)
            # print(targets.shape)
            # loss = criterion(output, targets)
            train_losses.append(loss.item())
            loss.backward()
            opt.step()
            
        # val_h = net.init_hidden(batch_size)
        val_losses = []
        accuracy=0
        f1score=0
        
        correct = 0
        total = 0
        total_true = []
        total_pred = []
        
        net.eval()
        with torch.no_grad():
            for batch in test_loader:
                x, y = batch
                inputs, targets = x.to(device), y.to(device)  
 
                # print(images.shape)            
            # for batch in iterate_minibatches(X_test, y_test, batch_size):
            #     x, y = batch     

                # inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

                # val_h = tuple([each.data for each in val_h])

                if(train_on_gpu):
                    inputs, targets = inputs.cuda(), targets.cuda()
                    
                output = net(inputs)

                # val_loss = criterion(output, torch.from_numpy(to_categorical(y, num_classes=NUM_CLASSES)).to(device))
                val_loss = criterion(output, torch.argmax(targets,dim=1))
                # val_loss = criterion(output, targets)
                val_losses.append(val_loss.item())
                
                predicted = torch.argmax(output.data, dim=1)
                total += targets.size(0)
                correct += (predicted == torch.argmax(targets, dim=1)).sum().item()

                total_pred = total_pred + predicted.cpu().numpy().tolist()
                total_true = total_true + (torch.argmax(targets, dim=1).cpu().numpy().tolist())


#                 top_p, top_class = output.topk(1, dim=1)
                
#                 # equals = top_class == torch.argmax(targets, dim=1)
#                 equals = top_class == targets.view(*top_class.shape).long()
#                 accuracy += torch.mean(equals.type(torch.FloatTensor))
#                 # f1score += metrics.f1_score(top_class.cpu(), torch.argmax(targets, dim=1).cpu(), average='micro')
#                 f1score += metrics.f1_score(top_class.cpu(), targets.view(*top_class.shape).long().cpu(), average='micro')
        net.train() # reset to train mode after iterationg through validation data
    
        # print(f'Test Accuracy: {100.0 * correct / total} %')
        # print(" | ".join(act_labels_txt))
        # conf_mat = confusion_matrix(y_true = total_true, y_pred = total_pred)
        # conf_mat = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
        # print(np.array(conf_mat).round(3) * 100)  
        f1_score = metrics.f1_score(y_true = total_true, y_pred = total_pred, average='weighted')
        # print('F1 score:', f1_score)
        # print('')      

        print("Epoch: {}/{}...".format(e+1, epochs),
        "Train Loss: {:.4f}...".format(np.mean(train_losses)),
        "Val Loss: {:.4f}...".format(np.mean(val_losses)),
        "Val Acc: {:.4f}...".format(correct / total),
        "F1-Score: {:.4f}...".format(f1_score))
        
        # PATH = 'opportunity_ConvAttn_ep'+str(e)+'.pt'
        # torch.save(net.state_dict(), PATH)
        
## check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

train(net)

Training on GPU!
Epoch: 1/600... Train Loss: 1.7882... Val Loss: 1.7870... Val Acc: 0.1840... F1-Score: 0.0580...
Epoch: 2/600... Train Loss: 1.7820... Val Loss: 1.7725... Val Acc: 0.3030... F1-Score: 0.1933...
Epoch: 3/600... Train Loss: 1.6323... Val Loss: 1.4911... Val Acc: 0.3544... F1-Score: 0.2855...
Epoch: 4/600... Train Loss: 1.4686... Val Loss: 1.4309... Val Acc: 0.4228... F1-Score: 0.3925...
Epoch: 5/600... Train Loss: 1.3921... Val Loss: 1.3472... Val Acc: 0.4038... F1-Score: 0.3926...
Epoch: 6/600... Train Loss: 1.3001... Val Loss: 1.1623... Val Acc: 0.5502... F1-Score: 0.5412...
Epoch: 7/600... Train Loss: 0.9763... Val Loss: 0.8949... Val Acc: 0.6737... F1-Score: 0.6753...
Epoch: 8/600... Train Loss: 0.8941... Val Loss: 0.9129... Val Acc: 0.6137... F1-Score: 0.6066...
Epoch: 9/600... Train Loss: 0.8432... Val Loss: 0.8784... Val Acc: 0.6583... F1-Score: 0.6312...
Epoch: 10/600... Train Loss: 0.7996... Val Loss: 0.7274... Val Acc: 0.7307... F1-Score: 0.7154...
Epoch: 11/60

In [9]:
# PATH = 'HHAR_Time_ConvAttn_2.pt'
# torch.save(net.state_dict(), PATH)